In [16]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, Listbox, MULTIPLE
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# 设置全局字体和显示配置
matplotlib.use('TkAgg')
plt.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = 'SimHei'

# 创建主窗口
root = tk.Tk()
root.title("热力图绘制工具")
root.geometry("1000x700")

# 默认全局变量
heatmap_title = "热力图"  # 默认热力图名称
data_numeric = None  # 用于保存处理后的数据
selected_columns = []  # 保存用户选择的列

def load_data(file_paths, file_format):
    global data_numeric

    all_data = []
    if file_paths:
        for file_path in file_paths:
            try:
                if file_format == 'Excel' and file_path.endswith('.xlsx'):
                    data = pd.read_excel(file_path)
                elif file_format == 'CSV' and file_path.endswith('.csv'):
                    data = pd.read_csv(file_path, encoding='utf-8')
                else:
                    messagebox.showwarning("警告", f"{file_path} 格式不匹配，已跳过。")
                    continue
            except UnicodeDecodeError:
                data = pd.read_csv(file_path, encoding='gbk')

            # 只保留数值型列（去除非数值型列）
            data_numeric = data.select_dtypes(include=['number'])
            if not data_numeric.empty:
                all_data.append(data_numeric)
            else:
                messagebox.showwarning("警告", f"{file_path} 中不包含任何数值型列，已跳过。")
        
        if all_data:
            # 合并所有数据
            data_numeric = pd.concat(all_data, axis=0, ignore_index=True)
            update_column_list(data_numeric.columns)
            messagebox.showinfo("成功", "所有数据已加载，请选择列进行处理。")
        else:
            messagebox.showerror("错误", "未能加载任何有效的数据。")
            data_numeric = None
    else:
        messagebox.showwarning("警告", "未选择任何文件。")

def update_column_list(columns):
    listbox.delete(0, tk.END)
    for col in columns:
        listbox.insert(tk.END, col)

def draw_heatmap():
    global data_numeric, selected_columns, heatmap_title

    selected_indices = listbox.curselection()
    selected_columns = [listbox.get(i) for i in selected_indices]
    
    if data_numeric is not None and selected_columns:
        # 过滤选择的列
        data_filtered = data_numeric[selected_columns]

        # 绘制热力图
        plt.figure(figsize=(12, 10))
        sns.heatmap(data_filtered, annot=True, cmap='coolwarm', fmt='.2f')

        # 设置标题和标签
        plt.title(heatmap_title)
        plt.xlabel('Columns')
        plt.ylabel('Rows')

        # 将图形嵌入到Tkinter的窗口中
        canvas = FigureCanvasTkAgg(plt.gcf(), master=root)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        messagebox.showinfo("成功", "热力图已绘制。")
    else:
        messagebox.showwarning("警告", "请先加载数据并选择列。")

def save_address():
    # 弹出文件保存对话框
    save_path = filedialog.asksaveasfilename(title="保存文件", defaultextension=".png",
                                             filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    if save_path:
        try:
            plt.savefig(save_path)  # 保存当前的热力图
            messagebox.showinfo("成功", f"文件已保存到: {save_path}")
        except Exception as e:
            messagebox.showerror("错误", f"保存文件时出错: {e}")
    else:
        messagebox.showwarning("警告", "未选择保存路径。")

def set_heatmap_title():
    global heatmap_title  # 使用全局变量来保存标题
    title = simpledialog.askstring("输入热力图名称", "请输入热力图名称：")
    if title:
        heatmap_title = title
        messagebox.showinfo("成功", f"热力图名称已设置为: {heatmap_title}")

def close_app():
    root.quit()  # 结束主循环
    root.destroy()  # 强制销毁窗口

def choose_files():
    # 让用户选择文件格式
    file_format = simpledialog.askstring("选择文件格式", "请输入文件格式（Excel或CSV）：")
    
    if file_format:
        file_format = file_format.strip().lower()
        if file_format == 'excel':
            file_format = 'Excel'
            file_types = [("Excel files", "*.xlsx")]
        elif file_format == 'csv':
            file_format = 'CSV'
            file_types = [("CSV files", "*.csv")]
        else:
            messagebox.showwarning("警告", "不支持的文件格式，请输入 'Excel' 或 'CSV'。")
            return

        # 根据用户选择的格式来过滤文件
        file_paths = filedialog.askopenfilenames(
            title="选择文件",
            filetypes=file_types
        )
        load_data(file_paths, file_format)
    else:
        messagebox.showwarning("警告", "未选择文件格式。")

# 创建一个Frame用于按键布局
button_frame = tk.Frame(root)
button_frame.pack(pady=20)

# 创建选择文件的按钮
choose_button = tk.Button(button_frame, text="选择文件", command=choose_files)
choose_button.grid(row=0, column=0, padx=10)

# 创建绘制热力图的按钮
draw_button = tk.Button(button_frame, text="绘制热力图", command=draw_heatmap)
draw_button.grid(row=0, column=1, padx=10)

# 创建保存地址的按钮
save_button = tk.Button(button_frame, text="保存热力图", command=save_address)
save_button.grid(row=0, column=2, padx=10)

# 创建自定义热力图名称的按钮
title_button = tk.Button(button_frame, text="设置热力图名称", command=set_heatmap_title)
title_button.grid(row=0, column=3, padx=10)

# 创建关闭应用程序的按钮
close_button = tk.Button(button_frame, text="关闭", command=close_app)
close_button.grid(row=0, column=4, padx=10)

# 列名列表框
listbox = Listbox(root, selectmode=MULTIPLE)
listbox.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

# 运行主循环
root.mainloop()
